In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision


import argparse
import sys


import numpy as np
import matplotlib.pyplot as plt

from data.EuroNotes import EuroNotes
from models.ResNet18 import pretrained_res18

In [2]:
# # # # # # # # # # # # # # # #
# Load Classifier Models
# # # # # # # # # # # # # # # #


print('Loading base classifier model and robust model')
model_base = pretrained_res18()
model_robust = pretrained_res18()
print('Loaded')

Loading base classifier model and robust model
Loaded


In [3]:
## From Alejandro
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=3, metavar='N',
                    help='number of epochs to train (default: 3)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=True,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args, unparsed = parser.parse_known_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()
args.epochs = 3
args.lr = 0.05

In [4]:
means = np.array([ 0.34065133, 0.30230788, 0.27947797])
stds = np.array([ 0.28919015, 0.26877816, 0.25182973])
transformations = transforms.Compose([transforms.ToTensor(), transforms.Normalize(means, stds)])

train_set = EuroNotes('../data-augmentation/banknotes_augmented/train', transform=transformations, resize=False)
val_set = EuroNotes('../data-augmentation/banknotes_augmented/val', transform=transformations, resize=False)

##Johannes's way..
train_loader = DataLoader(train_set, batch_size=25, shuffle=True, num_workers=16)
val_loader = DataLoader(val_set, batch_size=25, shuffle=True, num_workers=16)

##Alejandro's way..
#kwargs = {'num_workers': 4, 'pin_memory': True} if args.cuda else {}
#train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, **kwargs)
#val_loader = DataLoader(val_set, batch_size=args.batch_size, shuffle=True, **kwargs)

In [5]:
from attackers.WhiteBoxAttacker import PGDAttack

epsilon = 0.2 # Maximum strength of the perturbation
a = 0.01 # maximum change to the image
k = 40

pgd_attacker = PGDAttack(epsilon, a, k)

In [7]:
import matplotlib.pyplot as plt

for batch_idx, data in enumerate(train_loader):
    data_p, target = pgd_attacker.attack(model_base, data['image'], data['label'])
#     print(data[0,0,:,:])
#     print(data_p[0,0,:,:])
#     print(data_p[0,0,:,:]-Variable(data[0,0,:,:]))
#     break
    plt.subplot(1,2,1)
    plt.imshow(data_p[0,0,:,:].numpy()*0.3081+0.1307)
    plt.subplot(1,2,2)
    plt.imshow(data_p[0,0,:,:].data.numpy()*0.3081+0.1307)
    plt.show()
    break

/Users/d059191/anaconda/envs/anaconda2/lib/python2.7/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


AttributeError: 'Variable' object has no attribute 'numpy'

In [ ]:
import matplotlib.gridspec as gridspec

gs = gridspec.GridSpec(2, 2)
plt.figure(figsize=(20,10))

for i_batch, data in enumerate(train_loader):
    if i_batch > 3:
        break
    images = Variable(data['image'])
    labels = Variable(data['label'])
    preds = model_base(images)
    print(i_batch)
    print(preds)
    
    plt.title(i_batch)
    im = images[1, :, :, :].data.numpy() * stds.reshape((3, 1, 1))
    
    im = im + means.reshape((3, 1, 1))
    im = im.astype(float)
    im = im.astype(float)
    im -= np.min(im)
    im /= np.max(im)
    im = np.swapaxes(np.swapaxes(im, 0, 2).reshape((256, 256, 3)), 0, 1)
    ax = plt.subplot(gs[int(i_batch/2), i_batch % 2])
    plt.imshow(im)
    
plt.show()
    

In [ ]:
from attackers.BlackBoxAttacker import GANAttack

In [ ]:
ganatt = GANAttack((3, 256, 256), 0.01)

In [ ]:
images.shape

In [ ]:
preds = model_base(images)
print(preds.shape)

In [ ]:
np.random.uniform(-1, 0.1, (3, 2))